In [ ]:
!pip install findspark

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession, DataFrame, functions as F
from pyspark.sql.types import IntegerType,LongType,DoubleType,StringType

In [ ]:
name_file = '2022-11-04'

In [ ]:
!hdfs dfs -ls /user/ubuntu/data

In [ ]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
        .builder
        .appName("OTUS")
        .getOrCreate()
)

In [ ]:
spark.stop()

In [ ]:
path = "/user/ubuntu/data/2022-11-04.txt"

In [ ]:
path = "/user/ubuntu/data/2022-10-05.txt"

In [ ]:
path = "/user/ubuntu/data/*.txt"

In [ ]:
df = spark.read.text(path)

split_col = F.split(df['value'], ',')


df_split = (df
            .withColumn('tranaction_id', split_col.getItem(0))
            .withColumn('tx_datetime', split_col.getItem(1))
            .withColumn('customer_id', split_col.getItem(2))
            .withColumn('terminal_id', split_col.getItem(3))
            .withColumn('tx_amount', split_col.getItem(4))
            .withColumn('tx_time_seconds', split_col.getItem(5))
            .withColumn('tx_time_days', split_col.getItem(6))
            .withColumn('tx_fraud', split_col.getItem(7))
            .withColumn('tx_fraud_scenario', split_col.getItem(8))
            .drop(*['value'])
            .filter(~F.lower(F.col('tranaction_id')).like('%tranaction_id%'))
            .withColumn('date_key', F.to_date(F.col('tx_datetime').cast(StringType())))
            .withColumn('date_key', F.col('date_key').cast(StringType()))
            
#             .withColumn('hz', split_col.getItem(9))
#             .withColumn('hz2', split_col.getItem(10))
           )


df_split.show(3)

In [ ]:
df_split.printSchema()

In [ ]:
df_split.count()

In [ ]:
sdf_agg = (
    df_split
    .agg(
        F.max(F.col('tx_datetime')).alias('max_time'),
        F.min(F.col('tx_datetime')).alias('min_time'),
        F.max(F.col('date_key')).alias('max_date'),
        F.min(F.col('date_key')).alias('min_date'),
        )
    )

sdf_agg.show()

In [ ]:
sdf_agg = (
    df_split
    .agg(
        F.max(F.col('tx_datetime')).alias('max_time'),
        F.min(F.col('tx_datetime')).alias('min_time'),
        F.max(F.col('date_key')).alias('max_date'),
        F.min(F.col('date_key')).alias('min_date'),
        )
    )

sdf_agg.show()

In [ ]:
hdfs_path ='/user/ubuntu/data/'
name_file = 'clean_data'
output_hdfs_table_path = f'{hdfs_path}{name_file}.parquet'

output_hdfs_table_path
#mode= "overwrite"
mode ="append"
fmt= "parquet"
partition_cols= ("date_key",),
# output_hdfs_table_path

In [ ]:
sdf = df_split.sample(0.1)
sdf.printSchema()

(sdf
 .write
 .format(fmt)
 .mode(mode)
 .partitionBy(*partition_cols)
 .save(output_hdfs_table_path))



In [ ]:

print(output_hdfs_table_path)
df_from_parquet = spark.read.parquet(output_hdfs_table_path)

df_from_parquet.printSchema()

In [ ]:
sdf_agg = (
    df_split
    .agg(
        F.max(F.col('tx_datetime')).alias('max_date'),
        F.min(F.col('tx_datetime')).alias('min_date')
        )
    )

sdf_agg.show()

In [ ]:
df_split.sample(0.1).count()

In [ ]:
sdf: DataFrame,
    output_hdfs_table_path: str,
    output_hdfs_table_name: Optional[str] = None,
    partition_cols: Optional[Tuple[str]] = ("part_date",),
    num_out_partitions: Optional[int] = 1,
    fmt: str = "parquet",
    mode: str = "overwrite",

In [ ]:
(sdf.write.format(fmt).mode(mode).partitionBy(*partition_cols).save(output_hdfs_table_path))

In [ ]:
hdfs_path ='/user/ubuntu/data/'
name_file = '20221104'
output_hdfs_table_path = f'{hdfs_path}{name_file}.parquet'

output_hdfs_table_path
mode= "overwrite"
fmt= "parquet"
partition_cols= ("date_key",),
# output_hdfs_table_path

In [ ]:
sdf = df_split.sample(0.1)

(sdf
 .write
 .format(fmt)
 .mode(mode)
 .partitionBy(*partition_cols)
 .save(output_hdfs_table_path))




In [ ]:
sdf_agg = (
    df_split
    .select(*['tx_amount'])
    .dropna()
    .agg(
        F.round(F.expr("percentile(tx_amount, array(0.01))")[0], 4).alias("tx_amount_1perc"),
        F.round(F.expr("percentile(tx_amount, array(0.99))")[0], 4).alias("tx_amount_99perc"),
        )
    )

sdf_agg.printSchema()

In [ ]:
sdf_agg.show()

In [ ]:
col_list = [
    'tranaction_id',
    'tx_datetime',
    'customer_id',
    'terminal_id',
    'tx_amount',
    'tx_time_seconds',
    'tx_time_days',
    'tx_fraud',
    'tx_fraud_scenario'
]

sdf_clean = (
        df_split
        .join(sdf_agg, how='left')
        .filter(F.col('tx_amount')>0)
        .filter(F.col('tx_amount')>=F.col('tx_amount_1perc'))
        .filter(F.col('tx_amount')<=F.col('tx_amount_99perc'))
        .dropna()
        .filter(~F.col('terminal_id').rlike('[a-zA-Z]')) # только альфа-нумерик
        .filter(~F.col('customer_id').rlike('[a-zA-Z]')) # только альфа-нумерик
        .filter(~F.col('tranaction_id').rlike('[a-zA-Z]')) # только альфа-нумерик
        .filter(~F.col('tx_fraud').rlike('[a-zA-Z]')) # только альфа-нумерик
        .filter(~F.col('tx_fraud_scenario').rlike('[a-zA-Z]')) # только альфа-нумерик
        .filter(F.col('tx_time_seconds')>0) # только альфа-нумерик
        .filter(F.col('tx_time_days')>0) # только альфа-нумерик
        .select(*col_list)
        .distinct()
        .withColumn('tranaction_id', F.col('tranaction_id').cast(StringType()))
        .withColumn('tx_datetime', F.col('tx_datetime').cast(StringType()))
        .withColumn('customer_id', F.col('customer_id').cast(StringType()))
        .withColumn('terminal_id', F.col('terminal_id').cast(StringType()))
        .withColumn('tx_amount', F.col('tx_amount').cast(DoubleType()))
        .withColumn('tx_time_seconds', F.col('tx_time_seconds').cast(LongType()))
        .withColumn('tx_time_days', F.col('tx_time_days').cast(LongType()))
        .withColumn('tx_fraud', F.col('tx_fraud').cast(IntegerType()))
        .withColumn('tx_fraud_scenario', F.col('tx_fraud_scenario').cast(IntegerType()))
        .withColumn('part_date', F.lit(name_file))
)

sdf_clean.printSchema()

In [ ]:
#после удаления перцентилей
sdf_clean.count()

In [ ]:
46998983

In [ ]:
#после удаления пустых и текстовых и перцентилей
sdf_clean.count()

In [ ]:
#после удаления пустых и текстовых и перцентилей
sdf_clean.count()

In [ ]:
partition_cols = 'part_date'

In [ ]:
%%time

(
    sdf_clean
        .repartition(10)
        .write
        #.partitionBy(*partition_cols)
        .mode("overwrite")
        .parquet(output_hdfs_table_path)
        #.parquet("data/train.parquet")
)

In [ ]:
!hdfs dfs -ls -h data/train.parquet

In [ ]:
full_path = path_hdfs

In [ ]:
!hdfs dfs -ls /user/ubuntu/data

In [ ]:
hdfs_path ='/user/ubuntu/data/'
name_file = '20221104'
output_hdfs_table_path = f'{hdfs_path}{name_file}.parquet'
output_hdfs_table_path

In [ ]:
!hdfs dfs -ls /user/ubuntu/data

In [ ]:
!hdfs dfs -ls /user/ubuntu/data/20221104.parquet

In [ ]:
#проверка

In [ ]:
df_from_parquet = spark.read.parquet(output_hdfs_table_path)

df_from_parquet.printSchema()

In [ ]:
df_from_parquet.show(2)

In [ ]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)  # to pretty print pyspark.DataFrame in jupyter

In [ ]:
df_from_parquet.limit(10)

In [ ]:
df_from_parquet.count()

In [ ]:
!hdfs dfs -ls data

In [ ]:
!hdfs dfs -ls -h /user/ubuntu/data/20221104.parquet

In [ ]:
df_from_parquet.groupBy(['tx_fraud']).agg(F.sum(F.lit(1))).show()

In [ ]:
df_from_parquet.groupBy(['tx_fraud_scenario']).agg(F.sum(F.lit(1))).show()

In [ ]:
bucket_name = 'otus-bucket-b1gcs058gd4fa8eeviec'
input_path = f"s3a://{bucket_name}/*.txt"

In [ ]:
bucket_name = 'otus-bucket-b1gcs058gd4fa8eeviec'
input_path = f"s3a://{bucket_name}/*.txt"

df = spark.read.text(input_path)

split_col = F.split(df['value'], ',')


df_split = (df
            .withColumn('tranaction_id', split_col.getItem(0))
            .withColumn('tx_datetime', split_col.getItem(1))
            .withColumn('customer_id', split_col.getItem(2))
            .withColumn('terminal_id', split_col.getItem(3))
            .withColumn('tx_amount', split_col.getItem(4))
            .withColumn('tx_time_seconds', split_col.getItem(5))
            .withColumn('tx_time_days', split_col.getItem(6))
            .withColumn('tx_fraud', split_col.getItem(7))
            .withColumn('tx_fraud_scenario', split_col.getItem(8))
            .drop(*['value'])
            .filter(~F.lower(F.col('tranaction_id')).like('%tranaction_id%'))
            .withColumn('date_key', F.to_date(F.col('tx_datetime')))
            .withColumn('date_key', F.col('date_key').cast(StringType()))
            
            
#             .withColumn('hz', split_col.getItem(9))
#             .withColumn('hz2', split_col.getItem(10))
           )


df_split.show(3)

In [ ]:
df = spark.read.text(input_path)

In [ ]:
SHOW PARTITIONS customer;

In [ ]:
output_hdfs_table_path

In [ ]:
result_path = '/user/ubuntu/data/20221104.parquet'

In [ ]:
# df_from_parquet = spark.read.parquet(result_path)


print(output_hdfs_table_path)
df_from_parquet = spark.read.parquet(output_hdfs_table_path)

df_from_parquet.printSchema()

df_from_parquet2 = (df_from_parquet
                    .select('date_key')
                    .distinct()
                    .withColumn('date_key', F.col('date_key').cast(StringType()))
                   )

df_from_parquet2.printSchema()

In [ ]:
exist_partition = df_from_parquet2.select('date_key').distinct().collect()


In [ ]:
ex_partitions = [str(row[0]) for row in exist_partition]
    
ex_partitions

In [ ]:
# загрузим новый файл и проверим, что что-то есть

In [ ]:
bucket_name = 'otus-bucket-b1gcs058gd4fa8eeviec'
input_path = f"s3a://{bucket_name}/*.txt"

df = spark.read.text(input_path)

split_col = F.split(df['value'], ',')


df_split = (df
            .withColumn('tranaction_id', split_col.getItem(0))
            .withColumn('tx_datetime', split_col.getItem(1))
            .withColumn('customer_id', split_col.getItem(2))
            .withColumn('terminal_id', split_col.getItem(3))
            .withColumn('tx_amount', split_col.getItem(4))
            .withColumn('tx_time_seconds', split_col.getItem(5))
            .withColumn('tx_time_days', split_col.getItem(6))
            .withColumn('tx_fraud', split_col.getItem(7))
            .withColumn('tx_fraud_scenario', split_col.getItem(8))
            .drop(*['value'])
            .filter(~F.lower(F.col('tranaction_id')).like('%tranaction_id%'))
            .withColumn('date_key', F.to_date(F.col('tx_datetime')))
            .withColumn('date_key', F.col('date_key').cast(StringType()))
            
            
#             .withColumn('hz', split_col.getItem(9))
#             .withColumn('hz2', split_col.getItem(10))
           )


df_split.show(3)

df_split.printSchema()

In [ ]:
df_split.count()

In [ ]:
df_split.filter(~F.col('date_key').isin(ex_partitions)).count()

In [ ]:
sdf = df_split.filter(~F.col('date_key').isin(ex_partitions)).sample(0.1)
sdf.printSchema()

(sdf
 .write
 .format(fmt)
 .mode(mode)
 .partitionBy(*partition_cols)
 .save(output_hdfs_table_path))


In [ ]:
output_hdfs_table_path

In [ ]:

print(output_hdfs_table_path)
df_from_parquet = spark.read.parquet(output_hdfs_table_path)

df_from_parquet.printSchema()

df_from_parquet2 = (df_from_parquet
                    .select('date_key')
                    .distinct()
                    .withColumn('date_key', F.col('date_key').cast(StringType()))
                   )

df_from_parquet2.printSchema()

exist_partition = df_from_parquet2.select('date_key').distinct().collect()

ex_partitions_new = [str(row[0]) for row in exist_partition]

In [ ]:
ex_partitions_new 

In [ ]:
ex_partitions_new 

In [ ]:
df_from_parquet.count()

In [ ]:
print(93996170+46993904)

In [ ]:
append

In [ ]:
s3a://{bucket_name}

In [ ]:
!hdfs dfs -ls -h /user/ubuntu/data/20221104.parquet

In [ ]:
s3cmd --config=/home/ubuntu/.s3cfg ls s3://cold-s3-bucket/

In [ ]:
!s3cmd --config=/home/ubuntu/.s3cfg ls s3://cold-s3-bucket/

In [ ]:
bucket_name = 'otus-bucket-b1gcs058gd4fa8eeviec'
input_path = f"s3a://{bucket_name}/*.txt"



try:


    df = spark.read.text(input_path)

    split_col = F.split(df['value'], ',')


    partitions = (df
#                 .withColumn('tranaction_id', split_col.getItem(0))
                .withColumn('tx_datetime', split_col.getItem(1))
#                 .withColumn('customer_id', split_col.getItem(2))
#                 .withColumn('terminal_id', split_col.getItem(3))
#                 .withColumn('tx_amount', split_col.getItem(4))
#                 .withColumn('tx_time_seconds', split_col.getItem(5))
#                 .withColumn('tx_time_days', split_col.getItem(6))
#                 .withColumn('tx_fraud', split_col.getItem(7))
#                 .withColumn('tx_fraud_scenario', split_col.getItem(8))
#                 .drop(*['value'])
#                 .filter(~F.lower(F.col('tranaction_id')).like('%tranaction_id%'))
                .withColumn('date_key', F.to_date(F.col('tx_datetime')))
                .withColumn('date_key', F.col('date_key').cast(StringType()))
                .select('date_key')
                .distinct()
               ).collect()

    ex_partitions = [str(row[0]) for row in partitions]
except Exception:
    ex_partitions = []
    

In [ ]:
df_from_parquet = spark.read.parquet(output_hdfs_table_path)

partitions = (df_from_parquet
                    .select('date_key')
                    .distinct()
                    .withColumn('date_key', F.col('date_key').cast(StringType()))
                    .distinct()
                   ).collect()

ex_partitions = [str(row[0]) for row in partitions]

In [ ]:
ex_partitions

In [ ]:
python3 cleaning_data2.py --bucket 'otus-bucket-b1gcs058gd4fa8eeviec'

In [ ]:
df_from_parquet2 = (df_from_parquet
                    .select('date_key')
                    .distinct()
                    .withColumn('date_key', F.col('date_key').cast(StringType()))
                   )

In [ ]:
 s3a://otus-bucket-b1gcs058gd4fa8eeviec/input/*.txt

In [ ]:
bucket_name = 'otus-bucket-b1gcs058gd4fa8eeviec'
input_path = f"s3a://{bucket_name}/*.txt"

In [ ]:
path = "/user/ubuntu/data/2022-11-04.txt"

In [ ]:
df_from_parquet = spark.read.parquet(output_hdfs_table_path)

In [ ]:
bucket_name = 'cold-s3-bucket'
input_path = f"s3a://{bucket_name}/output_data/clean_data.parquet"


sdf = spark.read.parquet(input_path)

sdf.printSchema()
# try:


#     df = spark.read.text(input_path)

#     split_col = F.split(df['value'], ',')


#     partitions = (df
# #                 .withColumn('tranaction_id', split_col.getItem(0))
#                 .withColumn('tx_datetime', split_col.getItem(1))
# #                 .withColumn('customer_id', split_col.getItem(2))
# #                 .withColumn('terminal_id', split_col.getItem(3))
# #                 .withColumn('tx_amount', split_col.getItem(4))
# #                 .withColumn('tx_time_seconds', split_col.getItem(5))
# #                 .withColumn('tx_time_days', split_col.getItem(6))
# #                 .withColumn('tx_fraud', split_col.getItem(7))
# #                 .withColumn('tx_fraud_scenario', split_col.getItem(8))
# #                 .drop(*['value'])
# #                 .filter(~F.lower(F.col('tranaction_id')).like('%tranaction_id%'))
#                 .withColumn('date_key', F.to_date(F.col('tx_datetime')))
#                 .withColumn('date_key', F.col('date_key').cast(StringType()))
#                 .select('date_key')
#                 .distinct()
#                ).collect()

#     ex_partitions = [str(row[0]) for row in partitions]
# except Exception:
#     ex_partitions = []

In [ ]:
sdf.show(2)

In [ ]:
sdf.count()

In [ ]:
138174127*0.005

In [ ]:
df = sdf.sample(0.002).toPandas()

In [ ]:
df.head()

In [ ]:
df.to_csv('dt_test_sample.csv', index=True)

In [ ]:
df.tx_fraud.value_counts()

In [ ]:
bucket_name = 'cold-s3-bucket'
input_path = f"s3a://{bucket_name}/output_data/clean_data.parquet"


sdf = spark.read.parquet(input_path)

sdf.printSchema()
# try:


# agg 1 customer

In [ ]:
list_dates= sdf.select('date_key').distinct().collect()

In [ ]:
list_dates

In [ ]:
str(list_dates[0][0])

ld = [str(x[0]) for x in list_dates]

In [ ]:
start=min(ld)

In [ ]:
end=max(ld)

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
from tqdm import tqdm

In [ ]:
time_keys = [
    time_key.strftime("%Y-%m-%d") for time_key
    in pd.date_range(start, end, freq='1D')
]
time_keys

In [ ]:
for target_date in time_keys[20:21]:
    print(target_date)
    

In [ ]:
from tqdm.notebook import tqdm
from tqdm import tqdm

In [ ]:
deep = 4
sdf_cut_per = (sdf
                    .withColumn('target_date', F.lit(target_date))
                    .withColumn('target_date', F.to_date(F.col('target_date'), 'yyyy-MM-dd'))
                    .filter(F.col('date_key') >= F.date_sub(F.col('target_date'),deep))
                    .filter(F.col('date_key') <= F.date_sub(F.col('target_date'),1))             
                    )
sdf_cut_per.count()

In [ ]:
eps = 1e-6
deep = 4
bucket_name = 'cold-s3-bucket'
input_path = f"s3a://{bucket_name}/output_data/clean_data.parquet"


sdf = spark.read.parquet(input_path)

#for target_date in tqdm(time_keys[20:24]):
for target_date in tqdm(time_keys[24:26]):
    #print(target_date)

    sdf_cut_per = (sdf
                    .withColumn('target_date', F.lit(target_date))
                    .withColumn('target_date', F.to_date(F.col('target_date'), 'yyyy-MM-dd'))
                    .filter(F.col('date_key') >= F.date_sub(F.col('target_date'),deep))
                    .filter(F.col('date_key') <= F.date_sub(F.col('target_date'),1))             
                    )


    #sdf_cut_per.printSchema()

    sdf_customer = (sdf_cut_per
                    .groupBy(['customer_id','target_date'])
                    .agg(
                        F.countDistinct(F.col('date_key')).alias('cust_active_days_7d'),
                        F.countDistinct(F.col('terminal_id')).alias('cust_uniq_terminal_7d'),
                        #F.round(F.expr('percentile(tx_amount, 0.05)'), 0).alias('cust_amount_5perc'),
                        #F.round(F.expr('percentile(tx_amount, 0.25)'), 0).alias('cust_amount_25perc'),
                        F.round(F.expr('percentile(tx_amount, 0.5)'), 0).alias('cust_amount_50perc'),
                        #F.round(F.expr('percentile(tx_amount, 0.75)'), 0).alias('cust_amount_75perc'),
                        #F.round(F.expr('percentile(tx_amount, 0.95)'), 0).alias('cust_amount_95perc'),
                        F.max(F.col('tx_amount')).alias('cust_amount_max'),
                        F.min(F.col('tx_amount')).alias('cust_amount_min'),
                        F.sum(F.lit(1)).alias('cust_total_cnt_trans_7d'),                             
                        F.sum(
                            F.when(
                                (F.col('tx_fraud') == 1) 
                                , F.lit(1)).otherwise(F.lit(0)))
                                .alias('cust_bad_cnt_trans_7d'),
                        )


                    )

    #sdf_customer.printSchema()


    sdf_customer_per_day = (sdf_cut_per
                    .groupBy(['customer_id','target_date','date_key'])
                    .agg(
                        F.sum(F.col('tx_amount')).alias('sum_amount_in_day'),
                        F.sum(F.lit(1)).alias('cnt_in_day'),
                        F.sum(
                            F.when(
                                (F.col('tx_fraud') == 1) 
                                , F.lit(1)).otherwise(F.lit(0)))
                                .alias('bad_days'),

                        )
                    .withColumn('bad_days_ind', 
                        F.when(F.col('bad_days') > 0, F.lit(1))
                        .otherwise(F.lit(0)))
                    .groupBy(['customer_id','target_date'])
                    .agg(  
                        F.avg(F.col('sum_amount_in_day')).alias('cust_avg_amount_in_day_7d'),        
                        F.avg(F.col('cnt_in_day')).alias('cust_cnt_in_day_7d'), 
                        F.sum(F.col('bad_days_ind')).alias('cust_days_with_bad_trans_7d'),
                        )
                    )

    #sdf_customer_per_day.printSchema()


    sdf_customer_datamart = (sdf_customer
                            .join(sdf_customer_per_day, on=['customer_id','target_date'], how='inner') 
                            .withColumnRenamed('target_date', 'date_key') 
#                             .withColumn("rel_cust_5perc", (sdf_customer['cust_amount_5perc']-sdf_customer['cust_amount_min'])\
#                                / (sdf_customer['cust_amount_max'] - sdf_customer['cust_amount_min'] + eps))  
#                             .withColumn("rel_cust_25perc", (sdf_customer['cust_amount_25perc']-sdf_customer['cust_amount_min'])\
#                                / (sdf_customer['cust_amount_max'] - sdf_customer['cust_amount_min'] + eps))  
                            .withColumn("rel_cust_50perc", (sdf_customer['cust_amount_50perc']-sdf_customer['cust_amount_min'])\
                               / (sdf_customer['cust_amount_max'] - sdf_customer['cust_amount_min'] + eps)) 
#                             .withColumn("rel_cust_75perc", (sdf_customer['cust_amount_75perc']-sdf_customer['cust_amount_min'])\
#                                / (sdf_customer['cust_amount_max'] - sdf_customer['cust_amount_min'] + eps)) 
#                             .withColumn("rel_cust_95perc", (sdf_customer['cust_amount_95perc']-sdf_customer['cust_amount_min'])\
#                                / (sdf_customer['cust_amount_max'] - sdf_customer['cust_amount_min'] + eps)) 
                            )

    num_out_partitions=1
    
    if num_out_partitions:
        sdf_customer_datamart = sdf_customer_datamart.repartition(num_out_partitions)
    

    #sdf_customer_datamart.printSchema()
    
    bucket_name = 'cold-s3-bucket'
    output_table_path = f"s3a://{bucket_name}/output_data/agg_customer_data.parquet"
    
#     hdfs_path ='/user/ubuntu/data/'
#     name_file = 'agg_customer_data'
#     output_table_path = f'{hdfs_path}{name_file}.parquet'

    mode ="append"
    fmt= "parquet"
    partition_cols= ("date_key",),

    (sdf_customer_datamart
     .write
     .format(fmt)
     .mode(mode)
     .partitionBy(*partition_cols)
     .save(output_table_path))

In [ ]:
time_keys[20:24]

In [ ]:
time_keys[24:26]

In [ ]:
eps = 1e-6
deep = 4
bucket_name = 'cold-s3-bucket'
input_path = f"s3a://{bucket_name}/output_data/clean_data.parquet"


sdf = spark.read.parquet(input_path)

# for target_date in tqdm(time_keys[20:24]):
for target_date in tqdm(time_keys[24:26]):


    sdf_cut_per = (sdf
                    .withColumn('target_date', F.lit(target_date))
                    .withColumn('target_date', F.to_date(F.col('target_date'), 'yyyy-MM-dd'))
                    .filter(F.col('date_key') >= F.date_sub(F.col('target_date'),deep))
                    .filter(F.col('date_key') <= F.date_sub(F.col('target_date'),1))             
                    )


    #sdf_cut_per.printSchema()

    sdf_terminal = (sdf_cut_per
                    .groupBy(['terminal_id','target_date'])
                    .agg(
                        F.countDistinct(F.col('date_key')).alias('term_active_days_7d'),
                        F.countDistinct(F.col('customer_id')).alias('term_uniq_customer_7d'),
                        #F.round(F.expr('percentile(tx_amount, 0.05)'), 0).alias('term_amount_5perc'),
                        #F.round(F.expr('percentile(tx_amount, 0.25)'), 0).alias('term_amount_25perc'),
                        F.round(F.expr('percentile(tx_amount, 0.5)'), 0).alias('term_amount_50perc'),
                        #F.round(F.expr('percentile(tx_amount, 0.75)'), 0).alias('term_amount_75perc'),
                        #F.round(F.expr('percentile(tx_amount, 0.95)'), 0).alias('term_amount_95perc'),
                        F.sum(F.lit(1)).alias('term_total_cnt_trans_7d'),       
                        F.max(F.col('tx_amount')).alias('term_amount_max'),
                        F.min(F.col('tx_amount')).alias('term_amount_min'),
                        F.sum(
                            F.when(
                                (F.col('tx_fraud') == 1) 
                                , F.lit(1)).otherwise(F.lit(0)))
                                .alias('term_bad_cnt_trans_7d'),
                        )

                    )

    #sdf_terminal.printSchema()


    sdf_term_per_day = (sdf_cut_per
                    .groupBy(['terminal_id','target_date','date_key'])
                    .agg(
                        F.sum(F.col('tx_amount')).alias('sum_amount_in_day'),
                        F.sum(F.lit(1)).alias('cnt_in_day'),
                        F.sum(
                            F.when(
                                (F.col('tx_fraud') == 1) 
                                , F.lit(1)).otherwise(F.lit(0)))
                                .alias('bad_days'),

                        )
                    .withColumn('bad_days_ind', 
                        F.when(F.col('bad_days') > 0, F.lit(1))
                        .otherwise(F.lit(0)))
                    .groupBy(['terminal_id','target_date'])
                    .agg(  
                        F.avg(F.col('sum_amount_in_day')).alias('term_avg_amount_in_day_7d'),        
                        F.avg(F.col('cnt_in_day')).alias('term_cnt_in_day_7d'), 
                        F.sum(F.col('bad_days_ind')).alias('term_days_with_bad_trans_7d'),
                        )
                    )

    #sdf_term_per_day.printSchema()


    sdf_terminal_datamart = (sdf_terminal
                            .join(sdf_term_per_day, on=['terminal_id','target_date'], how='inner') 
                            .withColumnRenamed('target_date', 'date_key') 
                            )


    #sdf_terminal_datamart.printSchema()
    
    num_out_partitions=1
    
    if num_out_partitions:
        sdf_terminal_datamart = sdf_terminal_datamart.repartition(num_out_partitions)
    
    bucket_name = 'cold-s3-bucket'
    output_table_path = f"s3a://{bucket_name}/output_data/agg_term_data.parquet"
    
#     hdfs_path ='/user/ubuntu/data/'
#     name_file = 'agg_customer_data'
#     output_table_path = f'{hdfs_path}{name_file}.parquet'

    mode ="append"
    fmt= "parquet"
    partition_cols= ("date_key",),

    (sdf_terminal_datamart
     .write
     .format(fmt)
     .mode(mode)
     .partitionBy(*partition_cols)
     .save(output_table_path))

In [ ]:
import tqdm

In [ ]:
bucket_name = 'cold-s3-bucket'
output_table_path = f"s3a://{bucket_name}/output_data/agg_customer_data.parquet"

mode ="append"
fmt= "parquet"
partition_cols= ("date_key",),

(sdf
 .write
 .format(fmt)
 .mode(mode)
 .partitionBy(*partition_cols)
 .save(output_table_path))

In [ ]:
hdfs_path ='/user/ubuntu/data/'
name_file = 'agg_customer_data'
output_hdfs_table_path = f'{hdfs_path}{name_file}.parquet'

## сборка обучающей выборки

In [ ]:
time_keys[20:26]

In [ ]:
bucket_name = 'cold-s3-bucket'
row_path = f"s3a://{bucket_name}/output_data/clean_data.parquet"

row_sdf = spark.read.parquet(row_path)

row_sdf.printSchema()



agg_cust_path = f"s3a://{bucket_name}/output_data/agg_customer_data.parquet"

agg_cust_sdf = spark.read.parquet(agg_cust_path)

agg_cust_sdf.printSchema()


agg_term_path = f"s3a://{bucket_name}/output_data/agg_term_data.parquet"

agg_term_sdf = spark.read.parquet(agg_term_path)

agg_term_sdf.printSchema()


In [ ]:
target_date  = time_keys[23]
print(target_date)

In [ ]:
sdf.columns

In [ ]:
list_for_fillna = [
    'term_active_days_7d',
 'term_uniq_customer_7d',
 'term_amount_50perc',
 'term_total_cnt_trans_7d',
 'term_amount_max',
 'term_amount_min',
 'term_bad_cnt_trans_7d',
 'term_avg_amount_in_day_7d',
 'term_cnt_in_day_7d',
 'term_days_with_bad_trans_7d',
 'cust_active_days_7d',
 'cust_uniq_terminal_7d',
 'cust_amount_50perc',
 'cust_amount_max',
 'cust_amount_min',
 'cust_total_cnt_trans_7d',
 'cust_bad_cnt_trans_7d',
 'cust_avg_amount_in_day_7d',
 'cust_cnt_in_day_7d',
 'cust_days_with_bad_trans_7d',
 'rel_cust_50perc',
#  'sh_bad_trans_per_cust',
#  'sh_bad_trans_per_term',
#  'sh_bad_days_per_cust',
#  'sh_bad_days_per_term',
#  'rel_cust_amount_to_max',
#  'rel_term_amount_to_max'
    
    
    
]

In [ ]:
eps = 1e-6

target_date  = time_keys[23]

def datamart(date_loc):
    
    eps = 1e-6

    df = (row_sdf
               .filter(F.col('date_key') == date_loc)
               .join(agg_term_sdf, on=['terminal_id','date_key'], how='left')
               .join(agg_cust_sdf, on=['customer_id','date_key'], how='left')
               .fillna(value=0,subset=list_for_fillna)
               #.withColumn("sh_bad_trans_per_cust", sdf['cust_bad_cnt_trans_7d'] / (sdf.cust_total_cnt_trans_7d + eps))
    #            .withColumn("sh_bad_trans_per_term", sdf.term_bad_cnt_trans_7d / (sdf.term_total_cnt_trans_7d + eps))
              )

    #sdf.printSchema()

    df = (df


               .withColumn("sh_bad_trans_per_cust", df['cust_bad_cnt_trans_7d'] / (df['cust_total_cnt_trans_7d'] + eps))
               .withColumn("sh_bad_trans_per_term", df['term_bad_cnt_trans_7d'] / (df['term_total_cnt_trans_7d'] + eps))
               .withColumn("sh_bad_days_per_cust", df['cust_days_with_bad_trans_7d'] / (df['cust_active_days_7d'] + eps))
               .withColumn("sh_bad_days_per_term", df['term_days_with_bad_trans_7d'] / (df['term_active_days_7d'] + eps))
               .withColumn("sh_bad_trans_per_cust", df['cust_bad_cnt_trans_7d'] / (df['cust_total_cnt_trans_7d'] + eps))
               .withColumn("rel_cust_amount_to_max", (df['tx_amount']-df['cust_amount_min'])\
                           / (df['cust_amount_max'] - df['cust_amount_min'] + eps)) 
               .withColumn("rel_term_amount_to_max", (df['tx_amount']-df['term_amount_min'])\
                           / (df['term_amount_max'] - df['term_amount_min'] + eps))
               .withColumnRenamed('tx_fraud', 'target') 

               )


    #train_sdf.printSchema()
    return df

# sdf_mod.count()

In [ ]:
time_keys[24]

In [ ]:
train_sdf = datamart(time_keys[23])


In [ ]:
test_sdf = datamart(time_keys[24])

In [ ]:
train_sdf.printSchema()

In [ ]:
train_sdf.count()

In [ ]:
test_sdf.count()

In [ ]:
train_sdf.count()

In [ ]:
eps = 1e-6

target_date  = time_keys[24]

sdf_mod = (row_sdf
           .filter(F.col('date_key') == target_date)
           .join(agg_term_sdf, on=['terminal_id','date_key'], how='left')
           .join(agg_cust_sdf, on=['customer_id','date_key'], how='left')
           .fillna(value=0,subset=list_for_fillna)
           #.withColumn("sh_bad_trans_per_cust", sdf['cust_bad_cnt_trans_7d'] / (sdf.cust_total_cnt_trans_7d + eps))
#            .withColumn("sh_bad_trans_per_term", sdf.term_bad_cnt_trans_7d / (sdf.term_total_cnt_trans_7d + eps))
          )

sdf.printSchema()

train_sdf = (sdf_mod
            
            
           .withColumn("sh_bad_trans_per_cust", sdf_mod['cust_bad_cnt_trans_7d'] / (sdf_mod['cust_total_cnt_trans_7d'] + eps))
           .withColumn("sh_bad_trans_per_term", sdf_mod['term_bad_cnt_trans_7d'] / (sdf_mod['term_total_cnt_trans_7d'] + eps))
           .withColumn("sh_bad_days_per_cust", sdf_mod['cust_days_with_bad_trans_7d'] / (sdf_mod['cust_active_days_7d'] + eps))
           .withColumn("sh_bad_days_per_term", sdf_mod['term_days_with_bad_trans_7d'] / (sdf_mod['term_active_days_7d'] + eps))
           .withColumn("sh_bad_trans_per_cust", sdf_mod['cust_bad_cnt_trans_7d'] / (sdf_mod['cust_total_cnt_trans_7d'] + eps))
           .withColumn("rel_cust_amount_to_max", (sdf_mod['tx_amount']-sdf_mod['cust_amount_min'])\
                       / (sdf_mod['cust_amount_max'] - sdf_mod['cust_amount_min'] + eps)) 
           .withColumn("rel_term_amount_to_max", (sdf_mod['tx_amount']-sdf_mod['term_amount_min'])\
                       / (sdf_mod['term_amount_max'] - sdf_mod['term_amount_min'] + eps))
           .withColumnRenamed('tx_fraud', 'target') 
            
           )


train_sdf.printSchema()

# sdf_mod.count()

In [ ]:
numericColumns = list(
    map(
        lambda x: x[0], 
        filter(lambda x: x[1] == "double" or x[1] == "int", sdf.dtypes)
    )
)
numericColumns

In [ ]:
dft = sdf.withColumn(
    "target", 
    F.when(sdf["tx_fraud"] == 1, 1).otherwise(0)
)

dft.select("tx_fraud", "target").limit(5)

In [ ]:
dft.columns

In [ ]:
dft.select("tx_fraud", "target").show(5)

In [ ]:
from itertools import combinations

numericColumnsPairs = list(combinations(numericColumns, 2))
numericColumnsPairs

In [ ]:
%%time
corr = list(
    filter(
        lambda x: abs(x[2]) > 0.6, 
        map(lambda x: (x[0], x[1], dft.stat.corr(x[0], x[1])), numericColumnsPairs)
    )
)

In [ ]:
corr.sort(key=lambda x: x[2])
for i in reversed(corr):
    print(f"{i[0]:>25} {i[1]:>25}\t{i[2]}")

In [ ]:
numericColumnsFinal = list(
    set(numericColumns) - set(map(lambda x: x[1], corr))
)

numericColumnsFinal

In [ ]:
numericColumnsFinal =['term_amount_min',
 'term_amount_50perc',
 'term_amount_max',
 'tx_amount',
 'term_avg_amount_in_day_7d',
 'sh_bad_trans_per_cust',
 'cust_cnt_in_day_7d',
# 'tx_fraud',
 'rel_cust_50perc',
 'sh_bad_days_per_term',
 'rel_cust_amount_to_max']

In [ ]:
numericColumnsFinal

In [ ]:
# количество ноликов

In [ ]:
cnt_zeros=sdf.filter(F.col('tx_fraud')==0).count()
cnt_ones=sdf.filter(F.col('tx_fraud')==1).count()

In [ ]:
cnt_zeros

In [ ]:
cnt_ones

In [ ]:
from pyspark.ml.feature import VectorAssembler

featureColumns = numericColumnsFinal# + catColumns

assembler = VectorAssembler()\
    .setInputCols(featureColumns)\
    .setOutputCol("features")

assembled = assembler.transform(sdf)

In [ ]:
assembled.show(1)

In [ ]:
from pyspark.ml.feature import MinMaxScaler

scaler = MinMaxScaler()\
    .setInputCol("features")\
    .setOutputCol("scaledFeatures")

scaled = scaler.fit(assembled).transform(assembled)

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression()\
    .setMaxIter(1000)\
    .setRegParam(0.2)\
    .setElasticNetParam(0.8)\
    .setFamily("binomial")\
    .setFeaturesCol("scaledFeatures")\
    .setLabelCol("target")

lrModel = lr.fit(scaled)

print(f"Coefficients: {lrModel.coefficients}\nIntercept: {lrModel.intercept}")

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(
    labelCol = 'target',
    featuresCol = 'features',
    numTrees = 100
    )


# lr = LogisticRegression()\
#     .setMaxIter(1000)\
#     .setRegParam(0.2)\
#     .setElasticNetParam(0.8)\
#     .setFamily("binomial")\
#     .setFeaturesCol("features")\
#     .setLabelCol("target")


In [ ]:
model =rf.fit(assembled)

In [ ]:
from pyspark.ml import Pipeline

In [ ]:
assembler = VectorAssembler()\
    .setInputCols(featureColumns)\
    .setOutputCol("features")

scaler = MinMaxScaler()\
    .setInputCol("features")\
    .setOutputCol("scaledFeatures")

rf = RandomForestClassifier(
    labelCol = 'target',
    featuresCol = 'scaledFeatures',
    numTrees = 100
    )

In [ ]:
pipeline = Pipeline(stages = [assembler,scaler,rf])

In [ ]:
model = pipeline.fit(train_sdf)

In [ ]:
from pyspark.ml.functions import vector_to_array
th = 0.2
predictions = model.transform(test_sdf)

predictions = (predictions
              .withColumn('probability_arr', vector_to_array('probability'))
              .withColumn('probability_one', F.col('probability_arr')[1])
              .withColumn('pred_loc', 
                    F.when(F.col('probability_one') >= th, F.lit(1))
                    .otherwise(F.lit(0))) 
              )


    
    

In [ ]:
tp = predictions.filter((F.col("target") == 1) & (F.col("pred_loc") == 1)).count()
tn = predictions.filter((F.col("target") == 0) & (F.col("pred_loc") == 0)).count()
fp = predictions.filter((F.col("target") == 0) & (F.col("pred_loc") == 1)).count()
fn = predictions.filter((F.col("target") == 1) & (F.col("pred_loc") == 0)).count()

accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2*tp / (2*tp + fp +fn)
beta = 1.5
f_bet = (1+beta*beta)*tp / ((1+beta*beta)*tp+fp+beta*beta*fn)

print(f"Trashhold={th}")
print(f"Accuracy = {accuracy}")
print(f"Precision = {precision}")
print(f"Recall = {recall}")
print(f"f1 = {f1}")
print(f"f2 = {f_bet}")
print(f"Confusion Matrix:\n{tp:>4}\t{fp:>4}\n{fn:>4}\t{tn:>4}")
print()

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
evaluator = BinaryClassificationEvaluator()\
            .setLabelCol('target')

In [ ]:
auc = evaluator.evaluate(model.transform(test_sdf))

In [ ]:
print(auc)

In [ ]:
model.transform(test_sdf)

In [ ]:
predictions.columns

In [ ]:
predictions.select('scaledFeatures','prediction','target','probability').show(2)

In [ ]:
from pyspark.sql import types as T
from pyspark.sql.types import IntegerType,LongType,DoubleType,StringType,ArrayType

# Функция для опреледения cause
#@F.udf(returnType = T.StringType())
@F.udf(returnType = DoubleType())
def get_prob_ones(x):
    print(x)
    #return x[0][0][1]
    return x#[0][0][0]
    
#     if x in ('1', '2', '17', '18'):
#         return 'block'
#     elif x in ('8', '9', '21', '34', '42', '50', '55'):
#         return 'reruting'
#     elif x == 'unknown':
#         return 'unknown'
#     return 'other'

In [ ]:
a= predictions.select('probability').take(1)#.collect()

In [ ]:
predictions.count()

In [ ]:
1535752*0.0001

In [ ]:
type(a[0][0][0])

In [ ]:
a[0][0][1]

In [ ]:
a[0][0][0]

In [ ]:
split_col=F.split(F.col('probability'),",")

In [ ]:
from pyspark.sql.types import IntegerType,LongType,DoubleType,StringType,ArrayType,StructType,StructField

sheme = StructType([
    StructField('prob_zero',DoubleType()),
    StructField('prob_ones',DoubleType())
])

In [ ]:
predictions.printSchema()

In [ ]:
from pyspark.ml.feature import VectorSlicer

In [ ]:
slicer = VectorSlicer(inputCol = 'probability',outputCol = 'prob_one',indices =[1])

In [ ]:
predictions2 = slicer.transform(predictions)

In [ ]:
predictions2.select('probability','prob_one').show(2)

In [ ]:
from pyspark.ml.functions import vector_to_array

In [ ]:
predictions3 = (predictions2
                 .withColumn('prob_one2', vector_to_array('prob_one'))
                 .withColumn('prob_one3', F.col('prob_one2')[0])
#                 .withColumn('probe_one_class', F.transform(F.col('probability'), lambda x: x[0]))
                #.withColumn('probe_one_class', split_col.getItem(0))
                #.withColumn('probe_one_class', get_prob_ones(F.col('probability')))
                
#                .withColumn('prob_ones2', F.struct(F.col('probability'),'array'))
               
               )

In [ ]:
predictions3.select('prob_one2','prob_one','prob_one3').show(2)

In [ ]:
predictions3.printSchema()

In [ ]:
predictions2 = (predictions
                .sample(0.0001)
#                 .withColumn('probe_one_class', F.transform(F.col('probability'), lambda x: x[0]))
                #.withColumn('probe_one_class', split_col.getItem(0))
                #.withColumn('probe_one_class', get_prob_ones(F.col('probability')))
               #.withColumn('prob_ones', F.struct(F.col('probability'),'array'))
               
               )

In [ ]:
predictions2.select('prediction','target','probability','prob_ones').show(2)

In [ ]:
from pyspark.ml.

In [ ]:
tp = predictions.filter((F.col("target") == 1) & (F.col("prediction") == 1)).count()
tn = predictions.filter((F.col("target") == 0) & (F.col("prediction") == 0)).count()
fp = predictions.filter((F.col("target") == 0) & (F.col("prediction") == 1)).count()
fn = predictions.filter((F.col("target") == 1) & (F.col("prediction") == 0)).count()

print(f"Confusion Matrix:\n{tp:>4}\t{fp:>4}\n{fn:>4}\t{tn:>4}")

In [ ]:
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)

print(f"Accuracy = {accuracy}")
print(f"Precision = {precision}")
print(f"Recall = {recall}")

In [ ]:
fn

In [ ]:
predictions4 = predictions3

In [ ]:
treshholds = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]

for th in treshholds:
    predictions4 = (
        predictions3
        .withColumn('pred_loc', 
                F.when(F.col('prob_one3') >= th, F.lit(1))
                .otherwise(F.lit(0)))
    )
    
    tp = predictions4.filter((F.col("target") == 1) & (F.col("pred_loc") == 1)).count()
    tn = predictions4.filter((F.col("target") == 0) & (F.col("pred_loc") == 0)).count()
    fp = predictions4.filter((F.col("target") == 0) & (F.col("pred_loc") == 1)).count()
    fn = predictions4.filter((F.col("target") == 1) & (F.col("pred_loc") == 0)).count()
    
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2*tp / (2*tp + fp +fn)
    beta = 1.5
    f_bet = (1+beta*beta)*tp / ((1+beta*beta)*tp+fp+beta*beta*fn)
    
    print(f"Trashhold={th}")
    print(f"Accuracy = {accuracy}")
    print(f"Precision = {precision}")
    print(f"Recall = {recall}")
    print(f"f1 = {f1}")
    print(f"f2 = {f_bet}")
    print(f"Confusion Matrix:\n{tp:>4}\t{fp:>4}\n{fn:>4}\t{tn:>4}")
    print()
    

## clean

In [ ]:
train_sdf = datamart(time_keys[23])
test_sdf = datamart(time_keys[24])

In [ ]:
train_sdf.printSchema()

In [ ]:
numericColumnsFinal =['term_amount_min',
 'term_amount_50perc',
 'term_amount_max',
 'tx_amount',
 'term_avg_amount_in_day_7d',
 'sh_bad_trans_per_cust',
 'cust_cnt_in_day_7d',
# 'tx_fraud',
 'rel_cust_50perc',
 'sh_bad_days_per_term',
 'rel_cust_amount_to_max']

In [ ]:
sdf = train_sdf

In [ ]:
from pyspark.ml.feature import VectorAssembler

featureColumns = numericColumnsFinal# + catColumns

assembler = VectorAssembler()\
    .setInputCols(featureColumns)\
    .setOutputCol("features")

assembled = assembler.transform(sdf)

In [ ]:
from pyspark.ml.feature import MinMaxScaler

scaler = MinMaxScaler()\
    .setInputCol("features")\
    .setOutputCol("scaledFeatures")

scaled = scaler.fit(assembled).transform(assembled)

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(
    labelCol = 'target',
    featuresCol = 'features',
    numTrees = 100
    )

In [ ]:
model =rf.fit(assembled)

In [ ]:
from pyspark.ml import Pipeline

In [ ]:
assembler = VectorAssembler()\
    .setInputCols(featureColumns)\
    .setOutputCol("features")

scaler = MinMaxScaler()\
    .setInputCol("features")\
    .setOutputCol("scaledFeatures")

rf = RandomForestClassifier(
    labelCol = 'target',
    featuresCol = 'scaledFeatures',
    numTrees = 100
    )

pipeline = Pipeline(stages = [assembler,scaler,rf])

In [ ]:
model = pipeline.fit(train_sdf)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()\
            .setLabelCol('target')

auc = evaluator.evaluate(model.transform(test_sdf))

print(auc)

In [ ]:
from pyspark.ml.functions import vector_to_array
th = 0.2
predictions = model.transform(test_sdf)

predictions = (predictions
              .withColumn('probability_arr', vector_to_array('probability'))
              .withColumn('probability_one', F.col('probability_arr')[1])
              .withColumn('pred_loc', 
                    F.when(F.col('probability_one') >= th, F.lit(1))
                    .otherwise(F.lit(0))) 
              )

In [ ]:
tp = predictions.filter((F.col("target") == 1) & (F.col("pred_loc") == 1)).count()
tn = predictions.filter((F.col("target") == 0) & (F.col("pred_loc") == 0)).count()
fp = predictions.filter((F.col("target") == 0) & (F.col("pred_loc") == 1)).count()
fn = predictions.filter((F.col("target") == 1) & (F.col("pred_loc") == 0)).count()

accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2*tp / (2*tp + fp +fn)
beta = 1.5
f_bet = (1+beta*beta)*tp / ((1+beta*beta)*tp+fp+beta*beta*fn)

print(f"Trashhold={th}")
print(f"Accuracy = {accuracy}")
print(f"Precision = {precision}")
print(f"Recall = {recall}")
print(f"f1 = {f1}")
print(f"f2 = {f_bet}")
print(f"Confusion Matrix:\n{tp:>4}\t{fp:>4}\n{fn:>4}\t{tn:>4}")
print()

### hyper

In [ ]:
from functools import partial

In [ ]:
!pip install hyperopt

In [ ]:
from functools import partial

#import mlflow
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, SparkTrials, Trials

In [ ]:
# Определяем пространство поиска для hyperopt
search_space = {
    'numTrees': hp.randint('numTrees', 50, 150),
    'maxDepth': hp.randint('maxDepth', 3,7)
}

def objective(params, train_data, test_data):
    print(params)
    
    assembler = VectorAssembler()\
    .setInputCols(featureColumns)\
    .setOutputCol("features")

    scaler = MinMaxScaler()\
        .setInputCol("features")\
        .setOutputCol("scaledFeatures")
    
    rf = RandomForestClassifier()\
        .setFeaturesCol('scaledFeatures')\
        .setLabelCol('target')\
        .setMaxDepth(params['maxDepth'])\
        .setNumTrees(params['numTrees'])\

    pipeline = Pipeline(stages = [assembler,scaler,rf])

#     lr = LogisticRegression()\
#         .setMaxIter(1000)\
#         .setRegParam(params['regParam'])\
#         .setFeaturesCol('Features')\
#         .setLabelCol('Survived')

#     evaluator = BinaryClassificationEvaluator()\
#             .setLabelCol('Survived')

    rf_model = pipeline.fit(train_data)

    auc = evaluator.evaluate(rf_model.transform(test_data))

#     with mlflow.start_run():
#         mlflow.log_params(params)
#         mlflow.log_metric('auc', auc)
    
    return {'loss': -auc, 'status': STATUS_OK}

In [ ]:
trials = Trials()

# mlflow.set_experiment('classification')

best = fmin(
    fn=partial(
        objective, 
        train_data=train_sdf,
        test_data=test_sdf
    ),
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=trials
)

In [ ]:
best

In [ ]:
dft = train_sdf.withColumn(
    "target", 
    F.when(train_sdf["tx_fraud"] == 1, 1).otherwise(0)
)

dft.select("tx_fraud", "target").limit(5)

In [ ]:
!pip install loguru

In [ ]:
logging.basicConfig(level=logging.INFO, format="%(asctime)-15s %(message)s")
logger = logging.getLogger()

In [ ]:
import logger

In [ ]:
logger.info("Creating Spark Session ...")

In [ ]:
from loguru import logger

In [ ]:
def datamart(date_loc):
    
    eps = 1e-6

    df = (row_sdf
               .filter(F.col('date_key') == date_loc)
               .join(agg_term_sdf, on=['terminal_id','date_key'], how='left')
               .join(agg_cust_sdf, on=['customer_id','date_key'], how='left')
               .fillna(value=0,subset=list_for_fillna)
               #.withColumn("sh_bad_trans_per_cust", sdf['cust_bad_cnt_trans_7d'] / (sdf.cust_total_cnt_trans_7d + eps))
    #            .withColumn("sh_bad_trans_per_term", sdf.term_bad_cnt_trans_7d / (sdf.term_total_cnt_trans_7d + eps))
              )

    #sdf.printSchema()

    df = (df


               .withColumn("sh_bad_trans_per_cust", df['cust_bad_cnt_trans_7d'] / (df['cust_total_cnt_trans_7d'] + eps))
               .withColumn("sh_bad_trans_per_term", df['term_bad_cnt_trans_7d'] / (df['term_total_cnt_trans_7d'] + eps))
               .withColumn("sh_bad_days_per_cust", df['cust_days_with_bad_trans_7d'] / (df['cust_active_days_7d'] + eps))
               .withColumn("sh_bad_days_per_term", df['term_days_with_bad_trans_7d'] / (df['term_active_days_7d'] + eps))
               .withColumn("sh_bad_trans_per_cust", df['cust_bad_cnt_trans_7d'] / (df['cust_total_cnt_trans_7d'] + eps))
               .withColumn("rel_cust_amount_to_max", (df['tx_amount']-df['cust_amount_min'])\
                           / (df['cust_amount_max'] - df['cust_amount_min'] + eps)) 
               .withColumn("rel_term_amount_to_max", (df['tx_amount']-df['term_amount_min'])\
                           / (df['term_amount_max'] - df['term_amount_min'] + eps))
               .withColumnRenamed('tx_fraud', 'target') 

               )


    #train_sdf.printSchema()
    return df

search_space = {
    'numTrees': hp.randint('numTrees', 50, 150),
    'maxDepth': hp.randint('maxDepth', 3,7)
}

def objective(params, train_data, test_data):
    print(params)
    
    assembler = VectorAssembler()\
    .setInputCols(featureColumns)\
    .setOutputCol("features")

    scaler = MinMaxScaler()\
        .setInputCol("features")\
        .setOutputCol("scaledFeatures")
    
    rf = RandomForestClassifier()\
        .setFeaturesCol('scaledFeatures')\
        .setLabelCol('target')\
        .setMaxDepth(params['maxDepth'])\
        .setNumTrees(params['numTrees'])\

    pipeline = Pipeline(stages = [assembler,scaler,rf])

    rf_model = pipeline.fit(train_data)

    auc = evaluator.evaluate(rf_model.transform(test_data))

#     with mlflow.start_run():
#         mlflow.log_params(params)
#         mlflow.log_metric('auc', auc)
    
    return {'loss': -auc, 'status': STATUS_OK}

In [ ]:
bucket_name = 'cold-s3-bucket'
row_path = f"s3a://{bucket_name}/output_data/clean_data.parquet"

row_sdf = spark.read.parquet(row_path)

#row_sdf.printSchema()

agg_cust_path = f"s3a://{bucket_name}/output_data/agg_customer_data.parquet"
agg_cust_sdf = spark.read.parquet(agg_cust_path)
#agg_cust_sdf.printSchema()

agg_term_path = f"s3a://{bucket_name}/output_data/agg_term_data.parquet"
agg_term_sdf = spark.read.parquet(agg_term_path)
#agg_term_sdf.printSchema()  


list_for_fillna = [
'term_active_days_7d',
 'term_uniq_customer_7d',
 'term_amount_50perc',
 'term_total_cnt_trans_7d',
 'term_amount_max',
 'term_amount_min',
 'term_bad_cnt_trans_7d',
 'term_avg_amount_in_day_7d',
 'term_cnt_in_day_7d',
 'term_days_with_bad_trans_7d',
 'cust_active_days_7d',
 'cust_uniq_terminal_7d',
 'cust_amount_50perc',
 'cust_amount_max',
 'cust_amount_min',
 'cust_total_cnt_trans_7d',
 'cust_bad_cnt_trans_7d',
 'cust_avg_amount_in_day_7d',
 'cust_cnt_in_day_7d',
 'cust_days_with_bad_trans_7d',
 'rel_cust_50perc',
#  'sh_bad_trans_per_cust',
#  'sh_bad_trans_per_term',
#  'sh_bad_days_per_cust',
#  'sh_bad_days_per_term',
#  'rel_cust_amount_to_max',
#  'rel_term_amount_to_max'

]

list_dates= row_sdf.select('date_key').distinct().collect()
ld = [str(x[0]) for x in list_dates]
start=min(ld)
end=max(ld)
time_keys = [
    time_key.strftime("%Y-%m-%d") for time_key
    in pd.date_range(start, end, freq='1D')
]

train_sdf = datamart(time_keys[23])
test_sdf = datamart(time_keys[24])

numericColumnsFinal =['term_amount_min',
     'term_amount_50perc',
     'term_amount_max',
     'tx_amount',
     'term_avg_amount_in_day_7d',
     'sh_bad_trans_per_cust',
     'cust_cnt_in_day_7d',
    # 'tx_fraud',
     'rel_cust_50perc',
     'sh_bad_days_per_term',
     'rel_cust_amount_to_max']

In [ ]:
trials = Trials()

    # mlflow.set_experiment('classification')

best = fmin(
    fn=partial(
        objective, 
        train_data=train_sdf,
        test_data=test_sdf
    ),
    space=search_space,
    algo=tpe.suggest,
    max_evals=2,
    trials=trials
)

### clean 2

In [2]:
import findspark
findspark.init()

In [3]:
import os
from loguru import logger
from functools import partial

from pyspark.sql import SparkSession, DataFrame, functions as F
from pyspark.sql.types import IntegerType,LongType,DoubleType,StringType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.functions import vector_to_array
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql import types as T
from pyspark.sql.types import IntegerType,LongType,DoubleType,StringType,ArrayType
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, SparkTrials, Trials
#import mlflow
import pandas as pd

In [8]:

def datamart(date_loc):
    
    eps = 1e-6

    df = (row_sdf
               .filter(F.col('date_key') == date_loc)
               .join(agg_term_sdf, on=['terminal_id','date_key'], how='left')
               .join(agg_cust_sdf, on=['customer_id','date_key'], how='left')
               .fillna(value=0,subset=list_for_fillna)
               #.withColumn("sh_bad_trans_per_cust", sdf['cust_bad_cnt_trans_7d'] / (sdf.cust_total_cnt_trans_7d + eps))
    #            .withColumn("sh_bad_trans_per_term", sdf.term_bad_cnt_trans_7d / (sdf.term_total_cnt_trans_7d + eps))
              )

    #sdf.printSchema()

    df = (df


               .withColumn("sh_bad_trans_per_cust", df['cust_bad_cnt_trans_7d'] / (df['cust_total_cnt_trans_7d'] + eps))
               .withColumn("sh_bad_trans_per_term", df['term_bad_cnt_trans_7d'] / (df['term_total_cnt_trans_7d'] + eps))
               .withColumn("sh_bad_days_per_cust", df['cust_days_with_bad_trans_7d'] / (df['cust_active_days_7d'] + eps))
               .withColumn("sh_bad_days_per_term", df['term_days_with_bad_trans_7d'] / (df['term_active_days_7d'] + eps))
               .withColumn("sh_bad_trans_per_cust", df['cust_bad_cnt_trans_7d'] / (df['cust_total_cnt_trans_7d'] + eps))
               .withColumn("rel_cust_amount_to_max", (df['tx_amount']-df['cust_amount_min'])\
                           / (df['cust_amount_max'] - df['cust_amount_min'] + eps)) 
               .withColumn("rel_term_amount_to_max", (df['tx_amount']-df['term_amount_min'])\
                           / (df['term_amount_max'] - df['term_amount_min'] + eps))
               .withColumnRenamed('tx_fraud', 'target') 

               )


    #train_sdf.printSchema()
    return df


# Определяем пространство поиска для hyperopt
search_space = {
    'numTrees': hp.randint('numTrees', 50, 150),
    'maxDepth': hp.randint('maxDepth', 3,7)
}


def objective(params, train_data, test_data):
    logger.info(f"params {params}")

    
    assembler = VectorAssembler()\
    .setInputCols(featureColumns)\
    .setOutputCol("features")

    scaler = MinMaxScaler()\
        .setInputCol("features")\
        .setOutputCol("scaledFeatures")
    
    rf = RandomForestClassifier()\
        .setFeaturesCol('scaledFeatures')\
        .setLabelCol('target')\
        .setMaxDepth(params['maxDepth'])\
        .setNumTrees(params['numTrees'])\

    pipeline = Pipeline(stages = [assembler,scaler,rf])

    rf_model = pipeline.fit(train_data)
    
    evaluator = BinaryClassificationEvaluator()\
            .setLabelCol('target')

    auc = evaluator.evaluate(rf_model.transform(test_data))

#     with mlflow.start_run():
#         mlflow.log_params(params)
#         mlflow.log_metric('auc', auc)
    
    return {'loss': -auc, 'status': STATUS_OK}

In [7]:
logger.info("Creating Spark Session ...")
  
spark = SparkSession\
    .builder\
    .appName('Spark ML Research')\
    .config('spark.sql.repl.eagerEval.enabled', True) \
    .getOrCreate()

logger.info(spark)

bucket_name = 'cold-s3-bucket'
row_path = f"s3a://{bucket_name}/output_data/clean_data.parquet"

row_sdf = spark.read.parquet(row_path)

#row_sdf.printSchema()

agg_cust_path = f"s3a://{bucket_name}/output_data/agg_customer_data.parquet"
agg_cust_sdf = spark.read.parquet(agg_cust_path)
#agg_cust_sdf.printSchema()

agg_term_path = f"s3a://{bucket_name}/output_data/agg_term_data.parquet"
agg_term_sdf = spark.read.parquet(agg_term_path)
#agg_term_sdf.printSchema()  

logger.info("data upload ...")

list_for_fillna = [
'term_active_days_7d',
 'term_uniq_customer_7d',
 'term_amount_50perc',
 'term_total_cnt_trans_7d',
 'term_amount_max',
 'term_amount_min',
 'term_bad_cnt_trans_7d',
 'term_avg_amount_in_day_7d',
 'term_cnt_in_day_7d',
 'term_days_with_bad_trans_7d',
 'cust_active_days_7d',
 'cust_uniq_terminal_7d',
 'cust_amount_50perc',
 'cust_amount_max',
 'cust_amount_min',
 'cust_total_cnt_trans_7d',
 'cust_bad_cnt_trans_7d',
 'cust_avg_amount_in_day_7d',
 'cust_cnt_in_day_7d',
 'cust_days_with_bad_trans_7d',
 'rel_cust_50perc',
#  'sh_bad_trans_per_cust',
#  'sh_bad_trans_per_term',
#  'sh_bad_days_per_cust',
#  'sh_bad_days_per_term',
#  'rel_cust_amount_to_max',
#  'rel_term_amount_to_max'

]

list_dates= row_sdf.select('date_key').distinct().collect()
ld = [str(x[0]) for x in list_dates]
start=min(ld)
end=max(ld)
time_keys = [
    time_key.strftime("%Y-%m-%d") for time_key
    in pd.date_range(start, end, freq='1D')
]

logger.info(f"traind perion {time_keys[23]} test period {time_keys[24]}")

train_sdf = datamart(time_keys[23])
test_sdf = datamart(time_keys[24])

numericColumnsFinal =['term_amount_min',
     'term_amount_50perc',
     'term_amount_max',
     'tx_amount',
     'term_avg_amount_in_day_7d',
     'sh_bad_trans_per_cust',
     'cust_cnt_in_day_7d',
    # 'tx_fraud',
     'rel_cust_50perc',
     'sh_bad_days_per_term',
     'rel_cust_amount_to_max']
featureColumns = numericColumnsFinal

#mlflow.set_experiment('classification')

trials = Trials()

# mlflow.set_experiment('classification')

best = fmin(
    fn=partial(
        objective, 
        train_data=train_sdf,
        test_data=test_sdf
    ),
    space=search_space,
    algo=tpe.suggest,
    max_evals=2,
    trials=trials
)

2025-01-10 16:28:08.739 | INFO     | __main__:<module>:1 - Creating Spark Session ...
2025-01-10 16:28:17.982 | INFO     | __main__:<module>:9 - <pyspark.sql.session.SparkSession object at 0x7fb2423a4790>
2025-01-10 16:28:29.064 | INFO     | __main__:<module>:26 - data upload ...
2025-01-10 16:28:36.613 | INFO     | __main__:<module>:68 - traind perion 2022-09-28 test period 2022-09-29


  0%|          | 0/2 [00:00<?, ?trial/s, best loss=?]

2025-01-10 16:28:37.171 | INFO     | __main__:objective:45 - params {'maxDepth': 5, 'numTrees': 91}
job exception: name 'evaluator' is not defined



  0%|          | 0/2 [02:04<?, ?trial/s, best loss=?]


NameError: name 'evaluator' is not defined

In [6]:
spark.stop()